In [ ]:
import cv2
import pandas as pd
import os
import matplotlib.pyplot as plt
from ultralytics import YOLO
import numpy as np

# Path to the video file

In [ ]:
video_path = "path_to_your_video.mp4"

## Extracting Video Metadata

In [ ]:
def extract_video_metadata(video_path):
    video = cv2.VideoCapture(video_path)
    metadata = {
        "filename": os.path.basename(video_path),
        "duration": video.get(cv2.CAP_PROP_FRAME_COUNT) / video.get(cv2.CAP_PROP_FPS),
        "frame_rate": video.get(cv2.CAP_PROP_FPS),
        "frame_count": video.get(cv2.CAP_PROP_FRAME_COUNT),
        "width": video.get(cv2.CAP_PROP_FRAME_WIDTH),
        "height": video.get(cv2.CAP_PROP_FRAME_HEIGHT),
    }
    video.release()
    return metadata

# Extract and display metadata
metadata = extract_video_metadata(video_path)
print("Video Metadata:", metadata)

## Display Metadata as a DataFrame

In [ ]:
metadata_df = pd.DataFrame([metadata])
metadata_df

## Video Resolution Visualization

In [ ]:
plt.bar([metadata["filename"]], [metadata["width"] * metadata["height"]])
plt.ylabel("Resolution (pixels)")
plt.title("Video Resolution")
plt.show()

## Extracting and Visualizing a Frame

In [ ]:
def extract_frame(video_path, timestamp):
    video = cv2.VideoCapture(video_path)
    video.set(cv2.CAP_PROP_POS_MSEC, timestamp * 1000)
    success, frame = video.read()
    if success:
        frame_path = f"frame_at_{timestamp}s.jpg"
        cv2.imwrite(frame_path, frame)
        print(f"Frame extracted and saved at {frame_path}")
    video.release()
    return frame

frame = extract_frame(video_path, 5)
plt.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
plt.title("Frame at 5 seconds")
plt.axis("off")
plt.show()

## Playing Video Fragments

In [ ]:
def play_video_fragment(video_path, start_time, end_time):
    video = cv2.VideoCapture(video_path)
    frame_rate = int(video.get(cv2.CAP_PROP_FPS))
    start_frame = int(start_time * frame_rate)
    end_frame = int(end_time * frame_rate)

    video.set(cv2.CAP_PROP_POS_FRAMES, start_frame)

    while video.isOpened():
        ret, frame = video.read()
        if not ret or video.get(cv2.CAP_PROP_POS_FRAMES) > end_frame:
            break

        cv2.imshow("Video Fragment", frame)
        if cv2.waitKey(30) & 0xFF == ord('q'):
            break

    video.release()
    cv2.destroyAllWindows()

play_video_fragment(video_path, 5, 10)

## Object Detection Using YOLO

In [ ]:
model = YOLO("yolov8n.pt")  # Replace with the path to your YOLO model

def detect_objects_in_frame(frame):
    results = model(frame)
    return results[0].plot()  # Plot detections on the frame

annotated_frame = detect_objects_in_frame(frame)
plt.imshow(cv2.cvtColor(annotated_frame, cv2.COLOR_BGR2RGB))
plt.title("Objects Detected in Frame")
plt.axis("off")
plt.show()

## Detect Objects Throughout the Video

In [ ]:
def detect_objects_in_video(video_path):
    video = cv2.VideoCapture(video_path)
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_rate = int(video.get(cv2.CAP_PROP_FPS))
    detections = []

    for i in range(0, frame_count, frame_rate):  # Analyze 1 frame per second
        video.set(cv2.CAP_PROP_POS_FRAMES, i)
        success, frame = video.read()
        if not success:
            break
        results = model(frame)
        detections.append(len(results[0].boxes))  # Number of objects detected

    video.release()
    return detections

detections = detect_objects_in_video(video_path)
plt.plot(detections)
plt.xlabel("Frame Index")
plt.ylabel("Number of Objects")
plt.title("Objects Detected Per Frame")
plt.show()

## Generate Heatmap of Object Detection

In [ ]:
heatmap_data = np.zeros((int(metadata["height"] // 10), int(metadata["width"] // 10)))

video = cv2.VideoCapture(video_path)
for i in range(0, len(detections), 10):
    video.set(cv2.CAP_PROP_POS_FRAMES, i)
    success, frame = video.read()
    if not success:
        break
    results = model(frame)
    for box in results[0].boxes.xyxy:
        x1, y1, x2, y2 = box
        x1, x2 = int(x1 // 10), int(x2 // 10)
        y1, y2 = int(y1 // 10), int(y2 // 10)
        heatmap_data[y1:y2, x1:x2] += 1

video.release()
plt.imshow(heatmap_data, cmap="hot", interpolation="nearest")
plt.title("Object Detection Heatmap")
plt.axis("off")
plt.show()

## Saving Results

In [ ]:
average_objects = sum(detections) / len(detections)
print(f"Average objects detected per frame: {average_objects}")

output_df = pd.DataFrame({
    "frame_index": list(range(len(detections))),
    "objects_detected": detections
})
output_df.to_csv("video_eda_results.csv", index=False)
metadata_df.to_csv("video_metadata.csv", index=False)
print("EDA results saved to video_eda_results.csv and video_metadata.csv")